In [1]:
import pandas as pd
import requests
from io import StringIO, BytesIO
import sqlite3
import re
import os

# Google Drive file IDs extracted from share links
FILE_IDS = {
    'orders.csv': '1x-r51u5AFC4hJNYEiHNB4cIT707N7B9i',
    'users.json': '11DmwsptL8oct3bD3sk1NMzSsnv1kXajc',
    'restaurants.sql': '18iTTUvKOcWJvFUeIo2t7s2w2LtbJoTZR'
}


In [2]:

def download_gdrive_file(file_id, filename=None):
    """
    Download file from Google Drive public link by file ID.
    Returns file content as BytesIO for CSV/SQL, StringIO for text.
    """
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    response = requests.get(url)
    response.raise_for_status()

    # For large files, Google may show HTML page with download warning
    # Extract confirm token if present
    if 'content-disposition' not in response.headers and len(response.content) < 100000:
        m = re.search(r'"confirm=([0-9A-Za-z_]+)', response.text)
        if m:
            token = m.group(1)
            url += f"&confirm={token}"
            response = requests.get(url)
            response.raise_for_status()

    return BytesIO(response.content)


In [3]:

print("Step 1: Loading CSV Data (orders.csv)")
orders_content = download_gdrive_file(FILE_IDS['orders.csv'])


Step 1: Loading CSV Data (orders.csv)


In [4]:
orders = pd.read_csv(orders_content)
print(f"Orders shape: {orders.shape}")
print(orders.head())


Orders shape: (10000, 6)
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  


In [5]:
print("\nStep 2: Loading JSON Data (users.json)")
users_content = download_gdrive_file(FILE_IDS['users.json'])
users_content.seek(0)
users_text = users_content.read().decode('utf-8')
users = pd.read_json(StringIO(users_text))
print(f"Users shape: {users.shape}")


Step 2: Loading JSON Data (users.json)
Users shape: (3000, 4)


In [6]:
print(users.head(10))

   user_id     name       city membership
0        1   User_1    Chennai    Regular
1        2   User_2       Pune       Gold
2        3   User_3  Bangalore       Gold
3        4   User_4  Bangalore    Regular
4        5   User_5       Pune       Gold
5        6   User_6       Pune       Gold
6        7   User_7    Chennai       Gold
7        8   User_8  Hyderabad    Regular
8        9   User_9  Bangalore       Gold
9       10  User_10    Chennai    Regular


In [7]:
print("\nStep 3: Loading SQL Data (restaurants.sql)")
sql_content = download_gdrive_file(FILE_IDS['restaurants.sql'])
sql_content.seek(0)
sql_script = sql_content.read().decode('utf-8')


Step 3: Loading SQL Data (restaurants.sql)


In [8]:
# Create in-memory SQLite database
conn = sqlite3.connect(':memory:')
try:
    conn.executescript(sql_script)
    restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)  # Assume table name 'restaurants'
    print(f"Restaurants shape: {restaurants.shape}")
    print(restaurants.head())
finally:
    conn.close()

Restaurants shape: (500, 4)
   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [9]:
print("\nStep 4: Merging the Data (Left Joins on orders)")
# Merge orders with users on user_id
merged = orders.merge(users, on='user_id', how='left')

# Merge with restaurants on restaurant_id
final_df = merged.merge(restaurants, on='restaurant_id', how='left')

print(f"Final dataset shape: {final_df.shape}")


Step 4: Merging the Data (Left Joins on orders)
Final dataset shape: (10000, 12)


In [10]:
print(final_df.head())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3    Restaurant_224  Chinese    

In [11]:
print("\nStep 5: Saving Final Dataset")
final_df.to_csv('final_food_delivery_dataset.csv', index=False)
print("Saved to final_food_delivery_dataset.csv")


Step 5: Saving Final Dataset
Saved to final_food_delivery_dataset.csv


In [12]:
print("\nDataset ready for analysis:")
print("- Order trends: Analyze 'order_date' column")
print("- User behavior: Gold membership impact")
print("- City/cuisine performance: Group by city, cuisine")
print("- Revenue: Sum order amounts")
print(final_df.columns.tolist())



Dataset ready for analysis:
- Order trends: Analyze 'order_date' column
- User behavior: Gold membership impact
- City/cuisine performance: Group by city, cuisine
- Revenue: Sum order amounts
['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount', 'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y', 'cuisine', 'rating']


In [27]:
final_df.head(2)

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating,rating_range,quarter
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2,3.0-3.5,2023Q1
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5,4.1-4.5,2023Q1


In [14]:
# Q1: City with highest total revenue from Gold members
print("\n1️ City with highest Gold member revenue:")
gold_revenue = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].sum()
city_gold_max = gold_revenue.idxmax()
print(f"Answer: {city_gold_max}")



1️ City with highest Gold member revenue:
Answer: Chennai


In [15]:
# Q2: Cuisine with highest average order value
print("\n2️ Highest avg order value cuisine:")
cuisine_avg = final_df.groupby('cuisine')['total_amount'].mean().round(2)
cuisine_max = cuisine_avg.idxmax()
print(f"Answer: {cuisine_max}")


2️ Highest avg order value cuisine:
Answer: Mexican


In [16]:
# Q3: Distinct users with total orders > ₹1000
print("\n3️ Distinct high-value users:")
user_totals = final_df.groupby('user_id')['total_amount'].sum()
high_value_users = (user_totals > 1000).sum()
print(f"Answer: {high_value_users} users")


3️ Distinct high-value users:
Answer: 2544 users


In [17]:
# Q4: Restaurant rating range with highest revenue
print("\n4️ Highest revenue rating range:")
final_df['rating_range'] = pd.cut(final_df['rating'],
                                 bins=[0, 3.5, 4.0, 4.5, 5.0],
                                 labels=['3.0-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0'])
rating_revenue = final_df.groupby('rating_range')['total_amount'].sum()
rating_max = rating_revenue.idxmax()
print(f"Answer: {rating_max}")


4️ Highest revenue rating range:
Answer: 4.6-5.0


/tmp/ipython-input-925746903.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_revenue = final_df.groupby('rating_range')['total_amount'].sum()


In [36]:

# Q5: Gold members - highest avg order value city
print("\n5️ Gold members highest avg order city:")
gold_city_avg = final_df[final_df['membership'] == 'Gold'].groupby('city')['total_amount'].mean()
gold_city_max = gold_city_avg.idxmax()
print(f"Answer: {gold_city_max}")




5️ Gold members highest avg order city:
Answer: Chennai


In [37]:
# Q6: Cuisine with lowest distinct restaurants but significant revenue
print("\n6️ Lowest restaurants, high revenue cuisine:")
cuisine_stats = final_df.groupby('cuisine').agg({
    'restaurant_id': 'nunique',
    'total_amount': 'sum'
}).round(2)
# Sort by revenue desc, then restaurants asc
cuisine_rank = cuisine_stats.sort_values(['total_amount', 'restaurant_id'], ascending=[False, True])
print("Cuisine stats:")
print(cuisine_rank)
print(f"Answer: {cuisine_rank.index[0]}")




6️ Lowest restaurants, high revenue cuisine:
Cuisine stats:
         restaurant_id  total_amount
cuisine                             
Mexican            128    2085503.09
Italian            126    2024203.80
Indian             126    1971412.58
Chinese            120    1930504.65
Answer: Mexican


In [38]:
# Q7: Gold members % of total orders
print("\n7️ Gold members % of orders:")
total_orders = len(final_df)
gold_orders = len(final_df[final_df['membership'] == 'Gold'])
gold_pct = round((gold_orders / total_orders) * 100)
print(f"Answer: {gold_pct}%")



7️ Gold members % of orders:
Answer: 50%


In [39]:

# Q8: Restaurant with highest avg order value (<20 orders)
print("\n8️ Highest avg order restaurant (<20 orders):")
rest_orders = final_df.groupby('restaurant_name_x')['total_amount'].agg(['mean', 'count']).round(2)
rest_orders.columns = ['avg_amount', 'order_count']
low_volume = rest_orders[rest_orders['order_count'] < 20]
print(f"**{low_volume['avg_amount'].idxmax()}**")




8️ Highest avg order restaurant (<20 orders):
**Hotel Dhaba Multicuisine**


In [42]:
import pandas as pd

# Load dataset
df = pd.read_csv("final_food_delivery_dataset.csv")

# Given options
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]

# Filter dataset for only given options
df_opt = df[df["restaurant_name_x"].isin(options)]

# Calculate average order value and total orders
stats = df_opt.groupby("restaurant_name_x").agg(
    avg_order_value=("total_amount", "mean"),
    total_orders=("order_id", "count")
)

# Apply condition: less than 20 orders
valid = stats[stats["total_orders"] < 20]

# Pick highest average order value
answer = valid.sort_values(
    by="avg_order_value", ascending=False
).head(1)

print(answer)

                     avg_order_value  total_orders
restaurant_name_x                                 
Ruchi Foods Chinese       686.603158            19


In [45]:
# Given MCQ options
valid_options = [
    ("Gold", "Indian"),
    ("Gold", "Italian"),
    ("Regular", "Indian"),
    ("Regular", "Chinese")
]

# Filter to only those options
combo_revenue = final_df.groupby(['membership','cuisine'])['total_amount'].sum()
combo_filtered = combo_revenue.loc[valid_options]

# Pick the highest among the options
ans9 = combo_filtered.idxmax()
print("Q9: Which combination contributes the highest revenue (from given options)?")
print(f"Answer: {ans9[0]} + {ans9[1]} cuisine")


Q9: Which combination contributes the highest revenue (from given options)?
Answer: Gold + Italian cuisine


In [44]:

# Q10: Highest revenue quarter
print("\n Highest revenue quarter:")
final_df['order_date'] = pd.to_datetime(final_df['order_date'])
final_df['quarter'] = final_df['order_date'].dt.to_period('Q')
quarter_revenue = final_df.groupby('quarter')['total_amount'].sum()
quarter_max = quarter_revenue.idxmax()
print(f"Answer: {quarter_max}")



 Highest revenue quarter:
Answer: 2023Q3


In [49]:
# ANSWERS
print("1. Gold orders:", len(final_df[final_df['membership'] == 'Gold']))
print("2. Hyderabad revenue:", round(final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()))
print("3. Distinct users:", final_df['user_id'].nunique())
print("4. Gold avg:", round(final_df[final_df['membership'] == 'Gold']['total_amount'].mean(), 2))
print("5. Rating ≥4.5:", len(final_df[final_df['rating'] >= 4.5]))
print("6. Top Gold city:", len(final_df[(final_df['membership'] == 'Gold') & (final_df['city'] == 'Chennai')]))


1. Gold orders: 4987
2. Hyderabad revenue: 1889367
3. Distinct users: 2883
4. Gold avg: 797.15
5. Rating ≥4.5: 3374
6. Top Gold city: 1337
